In [ ]:
##Importing libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.externals import joblib
from sklearn.multioutput import MultiOutputClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
import matplotlib
import seaborn
import matplotlib.dates as md
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
#from pyemma import msm # not available on Kaggle Kernel
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
import os
os.environ["http_proxy"] = "http://one.proxy.att.com:8080"
os.environ["https_proxy"] = "http://one.proxy.att.com:8080"
!pip install sklearn_pandas

In [ ]:
#df = pd.read_csv('/home/jovyan/conf/master.csv')
df = pd.read_csv('/home/jovyan/conf/master.csv',error_bad_lines=False)
#df = pd.read_csv('CSI_Introscope_data.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
#df['METRIC_ATTRIBUTE']
for x in df.METRIC_ATTRIBUTE.unique():
    df[x]=(df.METRIC_ATTRIBUTE==x).astype(int)

In [ ]:
df['Average Response Time (ms)'] = df.VALUE_COUNT.values

In [ ]:
df['Average Response Time (ms)'].describe()

In [ ]:
#df.to_csv("aiops_csi_processed_data.csv", index=False)
df.to_csv("aiops_apm_processed_data.csv", index=False)

In [ ]:
#fill 0 for numbers and NNN for strings
for colname, col in df.iteritems():
    if(df[colname].dtype == np.object):
        df[colname] = df[colname].fillna('empty')
    else:
        df[colname] = df[colname].fillna(0) 

In [ ]:
df['date_time'] = pd.to_datetime(df['TS'])
df = df.sort_values('date_time')

In [ ]:
df['date_time'].min(), df['date_time'].max()

In [ ]:
categorical_columns=['Average Response Time (ms)']
continuous_columns=['MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']
feature_columns = categorical_columns + continuous_columns
output_columns=['Result']

In [ ]:
df = df[['date_time', 'Average Response Time (ms)', 'MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']]

In [ ]:
df = df.loc[df['Average Response Time (ms)'] < 1218]

In [ ]:
df['Average Response Time (ms)'].describe()

In [ ]:
# plot the data
df.plot(x='date_time', y='Average Response Time (ms)', figsize=(12,6))
plt.xlabel('Date time')
plt.ylabel('Average Response Time (ms)')
plt.title('Time Series of average response time by date time of search');

In [ ]:
df.head()

In [ ]:
a = df.loc[df['MIN_VALUE'] == 0, 'Average Response Time (ms)']
b = df.loc[df['MIN_VALUE'] == 1, 'Average Response Time (ms)']
plt.figure(figsize=(10, 6))
plt.hist(a, bins = 50, alpha=0.5, label='very min value')
plt.hist(b, bins = 50, alpha=0.5, label='min value')
plt.legend(loc='upper right')
plt.xlabel('Average Response Time')
plt.ylabel('Count')
plt.show();

In [ ]:
a = df.loc[df['MAX_VALUE'] == 0, 'Average Response Time (ms)']
b = df.loc[df['MAX_VALUE'] == 1, 'Average Response Time (ms)']
plt.figure(figsize=(10, 6))
plt.hist(a, bins = 50, alpha=0.5, label='very max value')
plt.hist(b, bins = 50, alpha=0.5, label='max value')
plt.legend(loc='upper right')
plt.xlabel('Average Response Time')
plt.ylabel('Count')
plt.show();

In [ ]:
a = df.loc[df['AGG_VALUE'] == 0, 'Average Response Time (ms)']
b = df.loc[df['AGG_VALUE'] == 1, 'Average Response Time (ms)']
plt.figure(figsize=(10, 6))
plt.hist(a, bins = 50, alpha=0.5, label='large agg value')
plt.hist(b, bins = 50, alpha=0.5, label='agg value')
plt.legend(loc='upper right')
plt.xlabel('Average Response Time')
plt.ylabel('AGG_VALUE')
plt.show();

In [ ]:
# Elbow Point Analysis - Kmeans
data = df[['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']]
n_cluster = range(1, 20)
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
scores = [kmeans[i].score(data) for i in range(len(kmeans))]
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(n_cluster, scores)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show();

In [ ]:
X = df[['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']]
X = X.reset_index(drop=True)
km = KMeans(n_clusters=7)
km.fit(X)
km.predict(X)
labels = km.labels_
#Plotting
fig = plt.figure(1, figsize=(7,7))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
ax.scatter(X.iloc[:,0], X.iloc[:,1], X.iloc[:,2],
          c=labels.astype(np.float), edgecolor="k")
ax.set_xlabel("Average Response Time (ms)")
ax.set_ylabel("VALUE_COUNT")
ax.set_zlabel("MIN_VALUE")
plt.title("K Means", fontsize=14);

In [ ]:
data = df[['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']]
X = data.values
X_std = StandardScaler().fit_transform(X)
#Calculating Eigenvecors and eigenvalues of Covariance matrix
mean_vec = np.mean(X_std, axis=0)
cov_mat = np.cov(X_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
# Create a list of (eigenvalue, eigenvector) tuples
eig_pairs = [ (np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

eig_pairs.sort(key = lambda x: x[0], reverse= True)

# Calculation of Explained Variance from the eigenvalues
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)] # Individual explained variance
cum_var_exp = np.cumsum(var_exp) # Cumulative explained variance

plt.figure(figsize=(10, 5))
plt.bar(range(len(var_exp)), var_exp, alpha=0.3, align='center', label='individual explained variance', color = 'g')
plt.step(range(len(cum_var_exp)), cum_var_exp, where='mid',label='cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.show();

In [ ]:
# Take useful feature and standardize them
data = df[['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']]
X_std = StandardScaler().fit_transform(X)
data = pd.DataFrame(X_std)
# reduce to 2 important features
pca = PCA(n_components=2)
data = pca.fit_transform(data)
# standardize these 2 new features
scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)

In [ ]:
df = df.sort_values('date_time')
df['date_time_int'] = df.date_time.astype(np.int64)

In [ ]:
kmeans = [KMeans(n_clusters=i).fit(data) for i in n_cluster]
df['cluster'] = kmeans[7].predict(data)
df.index = data.index
df['principal_feature1'] = data[0]
df['principal_feature2'] = data[1]
df['cluster'].value_counts()

In [ ]:
# return Series of distance between each point and its distance with the closest centroid
def getDistanceByPoint(data, model):
    distance = pd.Series()
    for i in range(0,len(data)):
        Xa = np.array(data.loc[i])
        Xb = model.cluster_centers_[model.labels_[i]-1]
        distance.set_value(i, np.linalg.norm(Xa-Xb))
    return distance

outliers_fraction = 0.01
# get the distance between each point and its nearest centroid. The biggest distances are considered as anomaly
distance = getDistanceByPoint(data, kmeans[7])
number_of_outliers = int(outliers_fraction*len(distance))
threshold = distance.nlargest(number_of_outliers).min()
# anomaly1 contain the anomaly result of the above method Cluster (0:normal, 1:anomaly) 
df['anomaly1'] = (distance >= threshold).astype(int)

In [ ]:
print(threshold)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
colors = {0:'blue', 1:'red'}
ax.scatter(df['principal_feature1'], df['principal_feature2'], c=df["anomaly1"].apply(lambda x: colors[x]))
plt.xlabel('principal feature1')
plt.ylabel('principal feature2')
plt.show();

In [ ]:
df.anomaly1.value_counts()

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

a = df.loc[df['anomaly1'] == 1, ['date_time_int', 'Average Response Time (ms)']] #anomaly

ax.plot(df['date_time_int'], df['Average Response Time (ms)'], color='blue', label='Normal')
ax.scatter(a['date_time_int'],a['Average Response Time (ms)'], color='red', label='Anomaly')
plt.xlabel('Date Time Integer')
plt.ylabel('Average Response Time (ms)')
plt.legend()
plt.show();

In [ ]:
a = df.loc[df['anomaly1'] == 0, 'Average Response Time (ms)']
b = df.loc[df['anomaly1'] == 1, 'Average Response Time (ms)']
fig, axs = plt.subplots(figsize=(10,6))
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'])
plt.show();

In [ ]:
#Isolation Forest
data = df[['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']]
#X = df[feature_columns]
scaler = StandardScaler()
np_scaled = scaler.fit_transform(data)
data = pd.DataFrame(np_scaled)
# train isolation forest
model =  IsolationForest(contamination=0.02)
model.fit(data)

df['anomaly2'] = pd.Series(model.predict(data))
# df['anomaly2'] = df['anomaly2'].map( {1: 0, -1: 1} )

fig, ax = plt.subplots(figsize=(10,6))

a = df.loc[df['anomaly2'] == -1, ['date_time_int', 'Average Response Time (ms)']] #anomaly

ax.plot(df['date_time_int'], df['Average Response Time (ms)'], color='blue', label = 'Normal')
ax.scatter(a['date_time_int'],a['Average Response Time (ms)'], color='red', label = 'Anomaly')
plt.legend()
plt.show();

In [ ]:
model

In [ ]:
df['anomaly2'].unique()

In [ ]:
# visualisation of anomaly with Average Response
a = df.loc[df['anomaly2'] == 1, 'Average Response Time (ms)']
b = df.loc[df['anomaly2'] == -1, 'Average Response Time (ms)']

fig, axs = plt.subplots(figsize=(10,6))
axs.hist([a,b], bins=32, stacked=True, color=['blue', 'red'])
plt.show();

In [ ]:
df['Average Response Time (ms)'].describe()

In [ ]:
#joblib.dump(model, "ai_ops_csi_v3.pkl")
joblib.dump(model, "aiops_apm.pkl")

In [ ]:
import pickle

# # Save to file in the current working directory
pkl_filename = "aiops_apm.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    saved_model = pickle.load(file)

In [ ]:
#df_test = pd.read_csv("aiops_csi_processed_response_data_test.csv")
#df_test = pd.read_csv("CSI_Introscope_data_Test.csv")
df_test = pd.read_csv("aiops_apm_testdata.csv")
#df_test = pd.read_csv("AvgResponseTime_Test.csv")

In [ ]:
df_test.shape

In [ ]:
for x in df_test.METRIC_ATTRIBUTE.unique():
    df_test[x]=(df_test.METRIC_ATTRIBUTE==x).astype(int)

In [ ]:
#Isolation Forest
#feature_columns = ['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']
df_test['Average Response Time (ms)'] = df_test['VALUE_COUNT'].copy()
data_test = df_test[feature_columns]
#output_columns = ['Average Response Time (ms)','MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT','Result']
scaler_test = StandardScaler()
np_scaled_test = scaler_test.fit_transform(data_test)
data_test = pd.DataFrame(np_scaled_test)
# # train isolation forest
results = pd.Series(saved_model.predict(data_test))
#df_test['results'] = results
df_test['anomaly2'] = results
outdf = df_test.merge(df_test['anomaly2'].to_frame(), left_index=True, right_index=True)
print(outdf)

In [ ]:
outdf

In [ ]:
# new, 'normal' observations ----
print("Accuracy:", list(results).count(1)/results.shape[0])
# Accuracy: 0.99

In [ ]:
%%writefile predict.py 
# Test Custom Predict File...
# you may import any libraries you need here

from sklearn.preprocessing import LabelEncoder 
from sklearn.multioutput import MultiOutputClassifier 
from cmlp.predictor import CMLPResponse, CMLPBasePredictor 
from sklearn.externals import joblib 
from io import BytesIO 
from io import BytesIO, StringIO 
import pandas as pd 
import numpy as np 
from itertools import zip_longest 
import json
from pandas.io.json import json_normalize
import json
from sklearn.preprocessing import StandardScaler

# This file must implement a Predictor class that extends the CMLPBasePredictor
class Predictor(CMLPBasePredictor):

    def predict(self, request):
        # your predict function must return a CMLPResponse object
        # you can return csv
        df_in = pd.read_csv(StringIO(request.data.decode()))
        categorical_columns=['Average Response Time (ms)']
        continuous_columns=['MIN_VALUE', 'MAX_VALUE','AGG_VALUE','VALUE_COUNT']
        feature_columns = categorical_columns + continuous_columns
        output_columns=['Result']
        for x in df_in.METRIC_ATTRIBUTE.unique():
            df_in[x]=(df_in.METRIC_ATTRIBUTE==x).astype(int)
            #print(df_in)
        #print(df_test)
        df_in = df_in.reset_index()
        df_in['Average Response Time (ms)'] = df_in['VALUE_COUNT'].copy()
        df_test = df_in[feature_columns]
        scaler = StandardScaler()
        np_scaled = scaler.fit_transform(df_test)
        df_test = pd.DataFrame(np_scaled)
        # # train isolation forest
        results = pd.Series(self.model.predict(df_test))
        #df_test['results'] = results
        df_test['anomaly2'] = results
        accuracy = list(results).count(1)/results.shape[0]
        print(accuracy)
        #df_test['accuracy'] = df_in.merge(df_test['accuracy'].to_frame(), left_index=True, right_index=True)
        #outdf = df_test.merge(df_test['anomaly2'].to_frame(), left_index=True, right_index=True)
        outdf = df_in.merge(df_test['anomaly2'].to_frame(), left_index=True, right_index=True)
        return CMLPResponse(outdf.to_csv(index=True), content_type='text/csv')
        #return CMLPResponse(outdf.to_csv(index=True), content_type='text/csv')
    def load(self, model):
        # you can load your model here. you may get the raw binary contents using 'model.contents'
        # this method will be called before predict() is called, and you must save this model as load is only called
        # once for a Predictor's lifecycle
        # example of loading an sklearn model using joblib:
        self.model = joblib.load(BytesIO(model.contents))

In [ ]:
%run predict.py

In [ ]:
from cmlp.predictor import CMLPModel, CMLPRequest

with open('aiops_apm_testdata.csv', 'rb') as input:
    cmlp_request = CMLPRequest(input.read(), 'text/csv')
predictor = Predictor()
with open('aiops_apm.pkl', 'rb') as model:
    predictor.load(CMLPModel(model.read()))
cmlp_response = predictor.predict(cmlp_request)
print(cmlp_response.data)